In [1]:
import requests
from hashlib import sha256

def APICall(difficulty: str, endpoint: str, uco:str, data: dict | None = None):
    url = f"https://ia174.fi.muni.cz/hw03/{difficulty}/{endpoint}/{uco}/"
    if data == None:
        request = requests.get(url)
    else:
        request = requests.post(url, json=data)
    
    if request.status_code != 200:
        print(f"Err {request.status_code}: {request.reason}")
        return None

    return  request.json()

def isGenerator(point, curve):
    factors = [x*y for (x, y) in curve.order().factor()]
    for factor in factors:
        if point * factor == curve.point((0, 1, 0)):
            return False
    if point * curve.order() == curve.point((0, 1, 0)):
        return True
    return False

n1 = 940258296925944608662895221235664431210
n2 = 42535295865117307932921825928971027169

gen_x = 23738974802862558648298862926228992519830411363340623631850893097053769277762
gen_y = 4527474540349461877783199522381786049485080389766292554055872436429664703463

p = 0x586be5268256ae12d62631efc2784d02dcff420d262da9cd94c62d5808bee24d
a = 0x7e7
b = 0x0

pub_x = 0x1a9112ae9ac6b30cb8f899f9ed76b4c3826b758d5c98a840afd9eee17d09fe73
pub_y = 0x39525bafcf33493141fd3f57d170f69af4ce41cd76ddb0f418781db2b4da5a8c

ecurve      = EllipticCurve(GF(p), [a, b])
generator   = ecurve.point((gen_x, gen_y))
pubkey      = ecurve.point((pub_x, pub_y))
order       = ecurve.order()
factors_all = [x^y for (x, y) in order.factor()]
factors     = factors_all[:-1]

print("Curve:")
print(ecurve)
print("Generator:")
print(generator)
print(isGenerator(generator, ecurve))
print("Public key:")
print(pubkey)
print(isGenerator(pubkey, ecurve))
print(order)
print(factors)


all_data = dict()
for factor in factors:
    #get random point with small order
    so_point = ecurve.point((0, 1, 0))
    print(f"\nFactor: {factor}")
    print(f"Order: {order}")
    while so_point.order() != factor:
        rng_point = ecurve.random_point()
        so_point = rng_point * (order // factor)
    
    print(f"Found point with order {factor} == {so_point.order()}: {so_point}")
    print(f"Generator: {isGenerator(so_point, ecurve)}")

    data = {"x": f"{hex(int(so_point[0]))}", "y": f"{hex(int(so_point[1]))}"}
    response = APICall("easy", "ecdh", "492875", data)
    if response == None:
        exit()

    secret = response["secret"]
    points = list()
    old_so_point = so_point
    for i in range(0, factor):
        points.append([so_point[0], so_point[1]])
        so_point += old_so_point
    all_data[factor] = [secret, points]

print(all_data)


Elliptic Curve defined by y^2 = x^3 + 2023*x over Finite Field of size 39994164849376373681302869279978885783653661374456939385898429379205375386189
(23738974802862558648298862926228992519830411363340623631850893097053769277762 : 4527474540349461877783199522381786049485080389766292554055872436429664703463 : 1)
True
(12016455826322587254171332197242588164952855197335449999871359431274478042739 : 25927346627286978519142378766841887155559783122479065444158612164844671621772 : 1)
False
39994164849376373681302869279978885783260159848204277050736846327351841544490
[2, 5, 13, 17, 29, 37, 41, 53, 61, 73, 89, 97, 101, 109, 113, 137, 149, 157, 173, 181, 193, 197]

Factor: 2
Order: 39994164849376373681302869279978885783260159848204277050736846327351841544490
Found point with order 2 == 2: (0 : 0 : 1)
Generator: False

Factor: 5
Order: 39994164849376373681302869279978885783260159848204277050736846327351841544490
Found point with order 5 == 5: (194884003089293389009201703119990151571786822312907615